In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

## First let us use the Logistic Regression in sklearn on Default dataset

In [2]:
df = pd.read_csv('Default.csv')
print(df.default.value_counts())
print(df.student.value_counts())
X = df[['balance','income']]
y = np.array([1 if (i =='Yes') else 0 for i in df['default'] ]).reshape((X.shape[0],))

No     9667
Yes     333
Name: default, dtype: int64
No     7056
Yes    2944
Name: student, dtype: int64


In [3]:
df.head(3)

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
#### Scale the features to make our result stable

#from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [5]:
### Black Box Logistic Regression with no penalty

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty = 'none',random_state=0)
model.fit(X_train,y_train)

model.coef_

model.intercept_

X_test = scaler.transform(X_test)

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[2880,    7],
       [  75,   38]])

## Some math behind Logistic Regression (gradient and hessian)

Define sigmoid funciton as:
$$\sigma(z) = \frac{1}{1+\exp(-z)} $$

The likelihood function: 
    $$\prod^{n}_{i=1} p(x_i)^{y_i} * (1-p(x_i))^{1-y_i} $$
where $$p(x_i) = \frac{1}{1+e^{-\sum(\beta_{j}*x_{ij})}}$$ where $$x_{i0} = 1$$

The Loss function is:
$$ l(\beta) = -\sum^{n}_{i=1}\left(y_ilog(\sigma(\beta^Tx_i)+(1-y_i)log(1-\sigma(\beta^Tx_i)\right)$$

Take derivative w.r.t. $\beta_j$ using chain rule:
$$\frac{\partial l}{\partial \beta_j} = \frac{\partial l}{\partial \sigma}  \frac{\partial \sigma}{\partial \beta^Tx_i}\frac{\partial \beta^Tx_i}{\partial \beta_j} $$

$$ = - \sum^n_{i=i} (y_i - \sigma(\beta^Tx_i))x_{ij} $$

In matrix format:
$$\frac{\partial l}{\partial \beta} = - X^T (y - \sigma (X\beta))$$

Hessian is the second derivative, therefore:
$$H = \frac{\partial l}{\partial \beta\beta^T} = X^T \frac{\partial \sigma}{\partial X\beta}X = X^TDX$$

where D is a n*n diagnal matrix such that:
$$ D_{ii} =  \sigma(\beta^Tx_i)(1- \sigma(\beta^Tx_i))$$

## Newton's method (which uses Hessian)

$$ \beta_{t+1} = \beta_{t} - H^{-1}(f(\beta_t))\nabla(f(\beta_t)) $$ 

In [6]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [7]:
iters = 10000
beta = np.zeros((3,1))
coef_vec = np.ones(len(X_train))

In [8]:
X_train = np.c_[coef_vec,X_train]

In [9]:
precision = 0.00001
loss_old = 1
for i in range(iters):
    # compute loss and the gradient
    loss = - (y_train.T @ np.log(sigmoid(X_train @ beta)) + (1-y_train.T) @ np.log(1-sigmoid(X_train @ beta)))
    gradient = X_train.T @ (sigmoid(X_train@beta)-y_train.reshape((-1,1)))
    
    # D is the diagnoal matrix in the middle of Hessian matrix Xt@D@X
    D = np.diag([float(sigmoid(beta.T@X_train[i,:].T)*(1-sigmoid(beta.T@X_train[i,:].T))) for i in range(len(X_train))])
    
    # compute hessian using D and X
    hessian = X_train.T @ D @ X_train
    
    # update step for Newton's method
    beta = beta - np.linalg.inv(hessian) @ gradient
    if abs(loss_old - loss) < precision:
        break
    else:
        loss_old = loss # store the loss and previous loss.
    print(loss)

[4852.03026392]
[1335.3488728]
[835.74356785]
[640.39047902]
[561.93210997]
[540.48959727]
[538.04848538]
[538.00381926]
[538.00380074]


In [10]:
beta # Coefficients obtained from our newton method

array([[-6.12225178],
       [ 2.66840103],
       [ 0.23775349]])

In [11]:
print(np.c_[model.intercept_,model.coef_].T) # Coefficients obtained from sklearn

[[-6.12225801]
 [ 2.66840546]
 [ 0.2377525 ]]


 Despite Newton's method requires intensive calculation (involving the hessian matrix), it converges quite fats given that initial starting values are zero for the coefficients. By just running for 9 iterations, this algorithm can achieve almost same coeficients as the black box results. 

## Minibach Gradient Descent with Size 64

In [12]:
gamma = 0.01 # exponentialy decaying learning rate
beta = np.zeros((3,1)) # initialize the 7 coefficients
iters = 10000
negLog_L_old=0
precision = 0.0001

In [14]:
for k in range(iters):
    negLog_L =  - (y_train.T @ np.log(sigmoid(X_train @ beta)) + (1-y_train.T) @ np.log(1.00001-sigmoid(X_train @ beta)))
    if pd.isna(negLog_L[0]):
        break
    idx = np.random.randint(7000,size=64)  ### size = 64
    beta = beta - gamma * np.exp(-0.0001*(k+1)) * -1 * X_train.T[:,idx] @ (y_train.reshape((-1,1))[idx] - sigmoid(X_train[idx,:]@beta))
    if np.abs(negLog_L_old-negLog_L) < precision:
        break
    else:
        negLog_L_old = negLog_L 
    print(negLog_L_old)

[538.91201646]
[538.66095027]
[538.89759757]
[538.96418003]
[538.85515543]
[539.22820813]
[538.65909429]
[538.64125767]
[539.24064044]
[539.3423008]
[539.20482366]
[538.9224852]
[539.21527071]
[538.98428288]
[539.31222551]
[540.87100164]
[540.74810421]
[540.17419484]
[540.84791399]
[540.68975628]
[539.96103867]
[539.26815225]
[538.76637304]
[538.68405094]
[538.50335278]
[539.79719912]
[541.33603299]
[539.74864055]
[540.20279406]
[540.83973817]
[541.88153615]
[540.67499391]
[540.41550631]
[541.52629748]
[539.77129371]
[540.67210372]
[541.27397533]
[540.69479387]
[541.48907989]
[538.45938973]
[538.60575663]
[538.15643005]
[538.21943768]
[538.11598117]
[538.13929716]
[538.03243455]
[538.04836573]
[538.31136135]
[538.30879546]
[537.94820491]
[537.95649374]
[538.32066112]
[537.94628875]
[538.24228803]
[538.25319735]
[538.23097337]
[538.02303205]
[537.98269354]
[538.1004378]
[539.22257443]
[538.27896492]
[538.046689]
[538.07444294]
[538.0371392]
[538.0496503]
[537.97069484]
[538.09336012]
[5

In [15]:
beta

array([[-6.18312068],
       [ 2.72024056],
       [ 0.1976446 ]])